In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import multiprocessing as mp
import pickle as pkl
import os
import re
from tqdm import tqdm
tqdm.pandas()
from IPython.display import display

# Deep Learning Imports 
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
kaggle = os.path.exists("/kaggle/input")
if kaggle:
    files = glob("../input/google-symptom-trends-as-of-october-1st-2022/202?_country_weekly_202?_US_weekly_symptoms_dataset.csv")
else:
    files = glob("datasets/202?_country_weekly_202?_US_weekly_symptoms_dataset.csv")
    from EDAModule.RegionVis import generalRegionVisualiztion

dfs = [pd.read_csv(file) for file in sorted(files)]
df = pd.concat(dfs, ignore_index=True)
del dfs

In [ ]:
# Data Stratification based on regions 
regions = df["sub_region_1_code"].unique()
regions = np.delete(regions, 0)
dfs = [df[df["sub_region_1_code"] == region].drop(columns=['sub_region_2', 'sub_region_2_code']) for region in regions]

# Store the weekly dataframes to a pickle seperate pickle files
for i, region in enumerate(regions):
    try:
        os.makedirs(f"./datasets/weekly/{region[3:]}")
    except FileExistsError:
        pass
    with open(f"./datasets/weekly/{region[3:]}/dataset.pkl", "wb") as f:
        pkl.dump(dfs[i], f)

del dfs

### Set Case: Georgia 

We will be using Georgia as our case study.

In [ ]:
f = open("./datasets/weekly/GA/dataset.pkl", "rb")
df = pkl.load(f)
symptoms = [col for col in df.columns if 'symptom' in col]
# df['sub_region_1_code']

In [ ]:
# Plot a missing data Seaborn heatmap fon Georgia dataset

ax, fig = plt.subplots(figsize=(20, 10))
sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.show()

Motherfucker, the dataset looks clean and actually better. 

In [ ]:
# Plot a distribution Seaborn heatmap for each symptom dataset for Georgia

ax, fig = plt.subplots(figsize=(20, 10))
sns.heatmap(df[symptoms], cmap="viridis", cbar=True)
plt.show()

In [ ]:
# Plot a correlation Seaborn heatmap for each symptom dataset for Georgia

ax, fig = plt.subplots(figsize=(20, 10))
sns.heatmap(df[symptoms].corr(), cmap="viridis")
plt.show()

In [ ]:
try:
    generalRegionVisualiztion(df, "./datasets/weekly/GA/")
except NameError:
    pass

In [ ]:
# Missing Data Analysis 

goodregions = []
for region in regions:
    df = pkl.load(open(f"./datasets/weekly/{region[3:]}/dataset.pkl", "rb"))
    # print(f"{region} has {df.isnull().sum().sum()} missing values")

    # Metric to assess the missing data
    print(f"{region[3:]} has {df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100}% missing values")

    # Get regions with good data
    if df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100 <= 0.01:
        goodregions.append(region[3:])

In [ ]:
### Empirical Statistical Analysis on the missingness in Google Symptom Data

# In each region get the number of missing values for each symptom

missingnessdf = pd.DataFrame()
for region in regions:
    df = pkl.load(open(f"./datasets/weekly/{region[3:]}/dataset.pkl", "rb"))
    df = df[symptoms]
    missingnessdf.loc[region[3:], symptoms] = df.isnull().sum().values

# Plot a heatmap of the missingness for each symptom in each region
ax, fig = plt.subplots(figsize=(20, 10))
sns.heatmap(missingnessdf, cmap="viridis")
plt.show()

### Missing Data checkpoint

Just saw the missing data heatmaps. Holy fuck, boy this is gonna be fun. There are missing data but not much. The missingness might have exagerrated by the resolution of daily datasets and might have increased as well. 

States with the most missing data: Alaska, Delaware, District of Columbia, Hawaii, Maine, Montana, New Hampshire, North Dakota, Rhode Island, South Dakota, Vermont, Wyoming.

With this consensus, the best way would be to train a model which have a better dataset like Florida, California, Georgia, Texas, New York and others which have a better dataset.

The popularity of the term would be conserved even if the differential privacy threshold doesn't hold. 
Using STRATS to impute the missing data from the other states would be a good idea.


In [ ]:
sym = 'symptom:Hemolysis'

# Load training data 
trainingRegions = goodregions
trainingData = [pkl.load(open(f"./datasets/weekly/{region}/dataset.pkl", "rb")).loc[:,['date',sym]].set_index('date') for region in trainingRegions]
trainingData = pd.concat(trainingData, axis = 1, ignore_index = False)
trainingData.columns = trainingRegions
trainingData = trainingData.transpose()



# Load testing data
testingData = pkl.load(open(f"./datasets/weekly/AR/dataset.pkl", "rb")).loc[:,['date',sym]]
masking_rate = testingData[sym].isnull().sum()/len(testingData)


# Plot the training and testing data
ax, fig = plt.subplots(figsize=(20, 10))
for region in trainingRegions:
    sns.lineplot(data=trainingData.loc[region], label=region)
# sns.lineplot(data=testingData, x="date", y=sym, label="Testing Data", hue = testingData[sym].isna().cumsum(), palette=["orange"]*sum(testingData[sym].isna()) + ["blue"]*(len(testingData) - sum(testingData[sym].isna())), legend=False, markers=True)
plt.show()

In [ ]:
# Making basic encoder and decoder model architecture 

class AE(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, latent_size):
        super(AE, self).__init__()
        self.input_size = input_size
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.hidden_size_3 = hidden_size_3
        self.hidden_size_4 = hidden_size_4
        self.latent_size = latent_size
        self.encoder = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size_1),
            nn.ReLU(),
            nn.Linear(self.hidden_size_1, self.hidden_size_2),
            nn.ReLU(),
            nn.Linear(self.hidden_size_2, self.hidden_size_3),
            nn.ReLU(), 
            nn.Linear(self.hidden_size_3, self.hidden_size_4),
            nn.ReLU(),
            nn.Linear(self.hidden_size_4, self.latent_size)
        )
        self.encoder_mu = nn.Linear(self.latent_size, self.latent_size)
        self.encoder_logvar = nn.Linear(self.latent_size, self.latent_size)
        self.decoder = nn.Sequential(
            nn.Linear(self.latent_size, self.hidden_size_4),
            nn.ReLU(),
            nn.Linear(self.hidden_size_4, self.hidden_size_3),
            nn.ReLU(),
            nn.Linear(self.hidden_size_3, self.hidden_size_2),
            nn.ReLU(),
            nn.Linear(self.hidden_size_2, self.hidden_size_1),
            nn.ReLU(),
            nn.Linear(self.hidden_size_1, self.input_size)
        )
    def forward(self, x):
        x = self.encoder(x)
        mu = self.encoder_mu(x)
        logvar = self.encoder_logvar(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
    
    def supervised_forward(self, x):
        x = self.encoder(x)
        mu = self.encoder_mu(x)
        logvar = self.encoder_logvar(x)
        z = self.reparameterize(mu, logvar)
        return mu, logvar, z
    
    def unsupervised_forward(self, x):
        x = self.encoder(x)
        mu = self.encoder_mu(x)
        logvar = self.encoder_logvar(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z)

In [ ]:
# Making a simple Non-linear Autoencoder model
input_size = trainingData.shape[1]

class FClayer(nn.Module):
    def __init__(self, input_size, output_size, activation = nn.ReLU()):
        super(FClayer, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation

        self.weight = nn.Parameter(torch.zeros(input_size, output_size))

        self.bias = nn.Parameter(torch.ones(1, output_size))

        torch.nn.init.xavier_uniform_(self.weight)
        torch.nn.init.xavier_uniform_(self.bias)

    def forward(self, x):
        output = torch.matmul(x,self.weight) + self.bias
        if self.activation:
            output = self.activation(output)
        return output

# Layers 

hidden_layer_1 = FClayer(input_size = input_size, output_size = 128, activation = nn.ReLU())
hidden_layer_2 = FClayer(input_size = 128, output_size = 64, activation = None)
hidden_layer_3 = FClayer(input_size = 64, output_size = 32, activation = nn.ReLU())
hidden_layer_4 = FClayer(input_size = 32, output_size = 16, activation = None)
hidden_layer_5 = FClayer(input_size = 16, output_size = 8, activation = nn.ReLU())
hidden_layer_6 = FClayer(input_size = 8, output_size = 4, activation = None)
hidden_layer_7 = FClayer(input_size = 4, output_size = 8, activation = nn.ReLU())
hidden_layer_8 = FClayer(input_size = 8, output_size = 16, activation = None)
hidden_layer_9 = FClayer(input_size = 16, output_size = 32, activation = nn.ReLU())
hidden_layer_10 = FClayer(input_size = 32, output_size = 64, activation = None)
hidden_layer_11 = FClayer(input_size = 64, output_size = 128, activation = nn.ReLU())
hidden_layer_12 = FClayer(input_size = 128, output_size = input_size, activation = None)

In [ ]:
# Source: https://github.com/shobrook/sequitur/blob/master/sequitur/models/lstm_ae.py

class EncoderRNN(nn.Module):
    def __init__(self, input_size, latent_size, hidden_size, hidden_activation, latent_activation):
        super(EncoderRNN, self).__init__()

        self.hidden_activation = hidden_activation
        self.latent_activation = latent_activation

        layer_size = [input_size] + hidden_size + [latent_size]
        self.num_layers = len(layer_size) - 1
        self.layers = nn.ModuleList()

        for i in range(self.num_layers):
            self.layers.append(nn.LSTM(input_size = layer_size[i], hidden_size = layer_size[i+1], num_layers = 1, batch_first=True))
        
    def forward(self, x):
        x = x.unsqueeze(0)
        for i, layer in enumerate(self.layers):
            x, (h_n, c_n) = layer(x)

            if self.hidden_activation and i < self.num_layers - 1:
                x = self.hidden_activation(x)
            elif self.latent_activation and i == self.num_layers - 1:
                return self.latent_activation(h_n).squeeze()
        
        return h_n.squeeze()



        
class DecoderRNN(nn.Module):
    def __init__(self, input_size, latent_size, hidden_size, hidden_activation, latent_activation):
        super(DecoderRNN, self).__init__()
        self.input_s = input_size
        self.latent_s = latent_size
        self.hidden_s = hidden_size

        self.hidden_activation = hidden_activation
        self.latent_activation = latent_activation

        layer_size = [input_size] + hidden_size + [hidden_size[-1]]
        self.num_layers = len(layer_size) - 1
        self.layers = nn.ModuleList()

        for i in range(self.num_layers):
            self.layers.append(nn.LSTM(layer_size[i], layer_size[i+1], num_layers = 1, batch_first=True))
        
        self.dense_matrix = nn.Parameter(torch.rand((layer_size[-1], latent_size), dtype = torch.float), requires_grad = True)  # latent_size == 1 for now or can use softmax layer


    def forward(self, x, seq_len):
        x = x.repeat(seq_len, 1).unsqueeze(0)
        for i, layer in enumerate(self.layers):
            x, (h_n, c_n) = layer(x)

            if self.hidden_activation and i < self.num_layers - 1:
                x = self.hidden_activation(x)
        
        return torch.matmul(x.squeeze(), self.dense_matrix).squeeze()

        

class LSTM_AE(nn.Module):
    def __init__(self, input_size, latent_size, hidden_size = [], hidden_activation = nn.Sigmoid(), latent_activation = nn.Tanh()):
        super(LSTM_AE, self).__init__()

        self.encoder = EncoderRNN(input_size, latent_size, hidden_size, hidden_activation, latent_activation)
        self.decoder = DecoderRNN(latent_size, input_size, hidden_size[::-1], hidden_activation, latent_activation)

    def forward(self, x):
        seq_len = x.shape[1]
        x = self.encoder(x)
        x = self.decoder(x, seq_len)
        return x# Source: https://github.com/shobrook/sequitur/blob/master/sequitur/models/lstm_ae.py

class Encoder(nn.Module):
    def __init__(self, input_size, latent_size, hidden_size, hidden_activation, latent_activation):
        super(Encoder, self).__init__()

        self.hidden_activation = hidden_activation
        self.latent_activation = latent_activation

        layer_size = [input_size] + hidden_size + [latent_size]
        self.num_layers = len(layer_size) - 1
        self.layers = nn.ModuleList()

        for i in range(self.num_layers):
            self.layers.append(nn.LSTM(input_size = layer_size[i], hidden_size = layer_size[i+1], num_layers = 1, batch_first=True))
        
    def forward(self, x):
        x = x.unsqueeze(0)
        for i, layer in enumerate(self.layers):
            x, (h_n, c_n) = layer(x)

            if self.hidden_activation and i < self.num_layers - 1:
                x = self.hidden_activation(x)
            elif self.latent_activation and i == self.num_layers - 1:
#                 print(f"Shape of x in encoder : {x.shape} and shape of latent h_n : {h_n.squeeze().shape}")
                return self.latent_activation(h_n).squeeze()
        return h_n.squeeze()



        
class Decoder(nn.Module):
    def __init__(self, input_size, latent_size, hidden_size, hidden_activation, latent_activation):
        super(Decoder, self).__init__()
        self.input_s = input_size
        self.latent_s = latent_size
        self.hidden_s = hidden_size

        self.hidden_activation = hidden_activation
        self.latent_activation = latent_activation

        layer_size = [input_size] + hidden_size + [hidden_size[-1]]
        self.num_layers = len(layer_size) - 1
        self.layers = nn.ModuleList()

        for i in range(self.num_layers):
            self.layers.append(nn.LSTM(layer_size[i], layer_size[i+1], num_layers = 1, batch_first=True))
#         print(f"Layer Size : {layer_size}")
        self.dense_matrix = nn.Parameter(torch.rand((layer_size[-1], latent_size), dtype = torch.float), requires_grad = True)

    def forward(self, x, seq_len):
        x = x.repeat(seq_len, 1).unsqueeze(0)
        for i, layer in enumerate(self.layers):
            x, (h_n, c_n) = layer(x)

            if self.hidden_activation and i < self.num_layers - 1:
                x = self.hidden_activation(x)
#         print(f"Shape of x in decoder : {x.squeeze(0).shape} and shape of matrix : {self.dense_matrix.shape}")
        return torch.mm(x.squeeze(0), self.dense_matrix)

        

class LSTM_AES(nn.Module):
    def __init__(self, input_size, latent_size, hidden_size = [], hidden_activation = nn.Sigmoid(), latent_activation = nn.Tanh()):
        super(LSTM_AES, self).__init__()

        self.encoder = Encoder(input_size, latent_size, hidden_size, hidden_activation, latent_activation)
        self.decoder = Decoder(latent_size, input_size, hidden_size[::-1], hidden_activation, latent_activation)

    def forward(self, x):
#         seq_len = x.shape[0]
        seq_len = 1
        
        x = self.encoder(x)
        x = self.decoder(x, seq_len)
#         print(f"Shape of LSTM module : {x.shape}")
        return x



In [ ]:
# Simple recurrent neural network

class RNN(nn.Module):
    def __init__(self) -> None:
        super().__init__(self, input_size, hidden_size, num_layers, num_classes)
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc =):

In [ ]:
# Model Initialization

input_size = trainingData.shape[1]
hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4 = 128, 64, 32, 16
latent_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AE(input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, latent_size).to(device)
# model = nn.Sequential(*[hidden_layer_1, hidden_layer_2, hidden_layer_3, hidden_layer_4, hidden_layer_5, hidden_layer_6, hidden_layer_7, hidden_layer_8, hidden_layer_9, hidden_layer_10, hidden_layer_10, hidden_layer_11, hidden_layer_12])
model = LSTM_AES(input_size, latent_size, [hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4]).to(device)
lr = 5e-5
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

In [ ]:
# Training the model
num_epochs = 200

min_loss = 100000
dfloss = pd.DataFrame(columns = ['epoch', 'loss'])
for epoch in tqdm(range(num_epochs)):
    model.train()
    # Randomize the order of training data
    trainingRegions = np.random.permutation(trainingRegions)
#     trainingData = trainingData.iloc[random_order]
    # Reduces learning rate every 50 epochs
    if not epoch % 50:
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr * (0.993 ** epoch)
    for region in trainingRegions:
        data = torch.tensor(trainingData.loc[region].values).float()
        data = data.to(device)
        # Mask some values as nan
        mask = torch.rand_like(data) < masking_rate

#         # Mask some values as nan and store them in a new tensor
#         mask = torch.rand_like(data) < masking_rate
        masked_data = data.clone()
        masked_data[mask] = float(0)
                

        # Forward pass
        output = model(masked_data)
#         output = model(mask.float() * data)
        # output = model(data)

        # Calculate the loss on the indexes where the values are not nan
        loss = criterion(output, data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if min_loss > loss.item():
        min_loss = loss.item()
        torch.save(model.state_dict(), "./LSTM_AE_onlymask.pth")
        print('Epoch [{}/{}], Loss:{:.7f}'.format(epoch + 1, num_epochs, loss.item()))
    dfloss = dfloss.append({'epoch': epoch, 'loss': loss.item()}, ignore_index=True)

ax, fig = plt.subplots(figsize = (20, 10))
sns.lineplot(x = 'epoch', y = 'loss', data = dfloss)
plt.show()


In [ ]:
# Load testing data
testingData = pkl.load(open(f"./datasets/weekly/AR/dataset.pkl", "rb")).loc[:,['date',sym]]
masking_rate = testingData[sym].isnull().sum()/len(testingData)

# Plot the predicted and actual data
model.load_state_dict(torch.load("./LSTM_AE.pth"))
model.eval()
output = model(torch.tensor(testingData[sym].transpose().fillna(0).values).float().to(device))
output = output.detach().numpy()

output.size

testingData['predicted'] = output.transpose()

# Substituting non nan values with actual values
# testingData.loc[~testingData[sym].isna(), 'predicted'] = testingData.loc[~testingData[sym].isna(), sym]

# Testing the various metrics
testingData['meanImpute'] = testingData[sym].fillna(value = testingData[sym].mean())
testingData['medianImpute'] = testingData[sym].fillna(value = testingData[sym].median())
testingData['forwardFill'] = testingData[sym].fillna(method = 'ffill')
testingData['backwardFill'] = testingData[sym].fillna(method = 'bfill')
testingData['interpolate'] = testingData[sym].interpolate(method = 'linear')

# Rolling mean imputation
testingData['rollingMeanImpute'] = testingData[sym].fillna(value = testingData[sym].rolling(5).mean())

testingData[sym] = testingData[sym].interpolate(method = 'krogh')

ax, fig = plt.subplots(figsize=(20, 10))
sns.lineplot(data=testingData, x="date", y=sym, label="Actual Data but krogh interpolated", legend=False, markers=True)
sns.lineplot(data=testingData, x="date", y='predicted', label="Predicted Data", legend=False, markers=True)
sns.lineplot(data=testingData, x="date", y='meanImpute', label="Mean Imputation", legend=False, markers=True)
sns.lineplot(data=testingData, x="date", y='medianImpute', label="Median Imputation", legend=False, markers=True)
sns.lineplot(data=testingData, x="date", y='forwardFill', label="Forward Fill", legend=False, markers=True)
sns.lineplot(data=testingData, x="date", y='backwardFill', label="Backward Fill", legend=False, markers=True)
# sns.lineplot(data=testingData, x="date", y='interpolate', label="Interpolation", legend=False, markers=True)
sns.lineplot(data=testingData, x="date", y='rollingMeanImpute', label="Rolling Mean Imputation", legend=False, markers=True)
plt.legend(loc='upper left')
plt.show()

from sklearn.metrics import mean_squared_error
testingData = testingData.dropna()
print(mean_squared_error(testingData[sym], testingData['predicted']))

### Latent Learning Two Cents

There is a lot of considerations we should keep in mind here.

1. Lack of training data 
2. Handling of masking and what we need to optimize
    > Elaboration on this: We need to predict missing data while my model is taking in entire masked data and then predicting actual data. 
    > Need help on this as to how this will work. 
3. I have used Linear Autoencoder and Fully Connected Autoencoder, which didn't work as expected. Fully Connected Autoencoder has better accuracy but than can be attributed to the fact that it had more layers. The gain in accuracy was marginal. 
4. Need to implement RNN techniques and most of it involves looking at the Attention Layer and understanding how it would help us. 
5. Masking is done on 0. We can preprocess the the data with the masking rate varying and then imputing it with the ffill and rfill methods. 

The imputation is 2 fronted problem now with the comparision and using LSTM to predict the missing data.

### Trend Visualization 

Lets see if the data joining worked and if we need to make adjustments for that in the early steps.

In [ ]:
# Plot cough, fever, hypoxemia symptoms for Georgia dataset
searchsymptoms = ["cough", "fever", "hypoxemia"]

# Load Georgia dataset
f = open("./datasets/weekly/Georgia/dataset.pkl", "rb")
df = pkl.load(f)

# Find columns which have cough, fever, and sore throat symptoms using regex search with ignoring case
symptoms = [col for col in df.columns if any(re.search(search, col, re.IGNORECASE) for search in searchsymptoms)]

# Plot the symptoms
ax, fig = plt.subplots(figsize=(20, 10))
for symptom in symptoms:
    plt.plot(df["date"], df[symptom], label=symptom)
plt.show()

## Canonical Correlation Analysis

### CCA

CCA is a multivariate analysis technique that is used to find linear relationships between two sets of variables. It is a generalization of the Pearson correlation coefficient, which is used to find the linear relationship between two sets of variables.


In [ ]:
if kaggle:
    df = pd.read_csv("../input/cdc-covid-tracker-dataset-for-us/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")
else:
    df = pd.read_csv("./datasets/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")

dfs = []
# Stratify the data by state
for region in df['state'].unique():
    statedf = df[df['state'] == region]
    statedf['date'] = pd.to_datetime(statedf['submission_date'])
    statedf = statedf.drop(['submission_date'], axis=1)
    statedf = statedf.resample('W-MON', on='date').sum()
    statedf = statedf.reset_index()
    # Get full 
    statedf['state'] = region
    try:
        pkl.dump(statedf, open(f"./datasets/weekly/{region}/CDCdataset.pkl", "wb"))
    except FileNotFoundError:
        pass
    dfs.append(statedf)

del dfs

In [ ]:
# For Georgia 

f = open(f"./datasets/weekly/GA/dataset.pkl", "rb")
df = pkl.load(f)
f = open(f"./datasets/weekly/GA/CDCdataset.pkl", "rb")
df2 = pkl.load(f)


df['date'] = pd.to_datetime(df['date'])

# Merge the datasets on date
df = df.merge(df2, on='date', how='inner')

df['date'] = pd.to_datetime(df['date'])

# Feature selection using correlation on the symptoms vs covid cases
symptoms = [col for col in df.columns if 'symptom' in col]

# Correlation analysis of the symptoms 
dfcorr = df[symptoms + ['new_case']].corr()
dfcorrcases = dfcorr.loc['new_case']

# Plot the correlation heatmap
plt.figure(figsize=(20, 10))
sns.lineplot(y = dfcorr.loc['new_case'].sort_values(), x = range(len(dfcorr.loc['new_case'])))
plt.show()

In [ ]:
# Get the Pearson correlation coefficient

from scipy.stats import pearsonr
from scipy.signal import correlate, correlation_lags

from statsmodels.tsa.stattools import grangercausalitytests

gransymsdf = pd.DataFrame({'symptom':[],'grangerCausalityPVal':[], 'mmcorrelation':[], 'time_lag':[], 'abscorrelation':[]})

# Get the granger causality test
for symptom in symptoms:
    grangerdat = df[['new_case', symptom]]

    # Standardize the data by columns
    grangerdatstd = (grangerdat - grangerdat.mean()) / grangerdat.std()
    
    # Normalize the data 
    grangerdatmm = (grangerdat - grangerdat.min()) / (grangerdat.max() - grangerdat.min())

    # Scipy cross correlation
    corrstd = correlate(grangerdatstd[symptom], grangerdatstd['new_case']) 
    lags = correlation_lags(len(grangerdat[symptom]), len(grangerdat['new_case']))
    corr = correlate(grangerdatmm[symptom], grangerdatmm['new_case'])
     
    for columns in grangerdat.columns:
        grangerdat[columns] = (grangerdat[columns] - grangerdat[columns].mean()) / grangerdat[columns].std()
        
    # Granger Test
    gran = grangercausalitytests(grangerdat[['new_case', symptom]], maxlag=2, verbose=False)
    
    gransymsdf = gransymsdf.append({'symptom':symptom,'grangerCausalityPVal':gran[1][0]['ssr_ftest'][1], 'mmcorrelation':np.max(corr), 'time_lag':time_lag, 'abscorrelation': np.max(corrstd)} , ignore_index = True)
        
gransymsdf.to_csv('featureselectionofsymptoms.csv')

Granger's Causality Test Assumption: Treats all symptoms as independent variables and the disease as the dependent variable.

In [ ]:
# Plot sin wave 
radians = np.linspace(0, 4*np.pi, 400)
rng = np.random.RandomState(12)
sin = np.sin(radians) + 0.1 * rng.randn(len(radians))
cos = np.cos(radians) + 0.1 * rng.randn(len(radians))
cos2 = 2*np.cos(radians) + 0.1 * rng.randn(len(radians))

# Min-max scaling 
sinmm = (sin - sin.min()) / (sin.max() - sin.min())
cosmm = (cos - cos.min()) / (cos.max() - cos.min())
cos2mm = (cos2 - cos2.min()) / (cos2.max() - cos2.min())

# Standardized scaling 
sinss = (sin - sin.mean()) / sin.std()
cosss = (cos - cos.mean()) / cos.std()
cos2ss = (cos2 - cos2.mean()) / cos2.std()

corr = correlate(sin, cos)/len(radians)
corr2 = correlate(sin, cos2)/len(radians)
corrmm = correlate(sinmm, cosmm)/len(radians)
corr2mm = correlate(sinmm, cos2mm)/len(radians)
corrss = correlate(sinss, cosss)/len(radians)
corr2ss = correlate(sinss, cos2ss)/len(radians)
lags = correlation_lags(len(sin), len(cos))

time_delay = lags[np.argmax(corr)]
time_delay2 = lags[np.argmax(corr2)]
time_delaymm = lags[np.argmax(corrmm)]
time_delay2mm = lags[np.argmax(corr2mm)]
time_delayss = lags[np.argmax(corrss)]
time_delay2ss = lags[np.argmax(corr2ss)]

plt.figure(figsize=(20, 10))
plt.plot(radians, sin, label='sin')
plt.plot(radians, cos, label='cos')
plt.axvline(radians[time_delay], color='red', label='time delay raw')
plt.axvline(radians[time_delay2], color='green', label='time delay raw 2')
plt.axvline(radians[time_delaymm], color='orange', label='time delay minmax')
plt.axvline(radians[time_delay2mm], color='purple', label='time delay minmax 2')
plt.axvline(radians[time_delayss], color='blue', label='time delay standardized')
plt.axvline(radians[time_delay2ss], color='black', label='time delay standardized 2')
plt.legend()
plt.show()

# Plot cross correlation

plt.figure(figsize=(20, 10))
plt.plot(lags, corr)
plt.plot(lags, corrmm)
plt.plot(lags, corrss)
plt.legend(['raw', 'minmax', 'standardized'])
plt.xlabel('lag')
plt.ylabel('correlation')
plt.show()

print(f'Time delay: {time_delay} \t Correlation: {np.max(corr), np.max(corr2)} \t Standardized: {np.max(corrss), np.max(corr2ss)} \t Min-Max: {np.max(corrmm), np.max(corr2mm)}')

In [ ]:
# Extract top 20 symptoms with highest absolute correlation
corrsyms = gransymsdf.sort_values(by='abscorrelation', ascending=False).head(20)['symptom'].values
mmcorrsyms = gransymsdf.sort_values(by='mmcorrelation', ascending=False).head(20)['symptom'].values

# Extract top 20 symptoms with highest granger causality
gransyms = gransymsdf.sort_values(by='grangerCausalityPVal', ascending=True).head(20)['symptom'].values

# Extract top 20 symptoms with highest leading time lag
gransymsdf['magdelay'] = gransyms['time_lag'].abs()
lagsyms = gransymsdf.sort_values(by = 'magdelay', ascending=True).head(20)['symptom'].values


In [ ]:
%%time
# Feature Selection using Random Forest Regressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(789)

# Features
features = [col for col in df.columns if 'symptom' in col]

# Target
target = 'new_case'

# Split the data to train and test
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=rng)

rf = RandomForestRegressor(random_state=rng)

# Random Hyperparameter Grid
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['auto']
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 5]
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=1, random_state=rng, n_jobs = -1, scoring='neg_root_mean_squared_error')

# Fit the random search model
rf_random.fit(X_train, y_train)

# Extract the best parameters
rf_random.best_params_

In [ ]:
from sklearn.metrics import mean_squared_error

def eval(model, test_features, test_labels):
    predictions = model.predict(test_features)
    rmse = mean_squared_error(test_labels, predictions, squared=False)
    print('Model Performance (RMSE) {}'.format(rmse))
    return rmse

base_model = RandomForestRegressor(n_estimators = 10, random_state = rng)
base_model.fit(X_train, y_train)
base_accuracy = eval(base_model, X_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = eval(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (-random_accuracy + base_accuracy) / base_accuracy))

In [ ]:
%%time
param_grid = {
    'bootstrap': [True],
    'max_depth': [None, 50, 60, 70],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [400, 500, 600, 700, 800]
}

# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 1, scoring='neg_root_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = eval(best_grid, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (-grid_accuracy + base_accuracy) / base_accuracy))

In [ ]:
%%time 
# Feature Selection using RFE-SVM and RFE-Linear Regression
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(789)

# Get the features
features = [col for col in df.columns if 'symptom' in col]

# Get the target
target = 'new_case'

# Get the features and target
X = df[features]
y = df[target]

# Split the data to train and test
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=rng)

# RFECV to get the total number of features
rfecv = RFECV(estimator=grid_search.best_estimator_, cv=5, verbose = 1, n_jobs = -1, scoring='neg_root_mean_squared_error')

# Fit the RFECV
rfecv.fit(X_train, y_train)

# Get the number of features
print(f'Optimal number of features: {rfecv.n_features_}')

# Plot the number of features vs. cross-validation scores
plt.figure(figsize=(20, 10))
plt.xlabel('Number of features selected')
plt.ylabel('Cross validation score (nb of correct classifications)')
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'])
plt.show()

# Get the selected features
rfsyms = df[features].columns[rfecv.support_]

Forecasting Effect of the Feature Selected Symptoms on the Cases

In [ ]:
# Feature Selection Study Results

# DARTS Implementation
try:
    from darts import TimeSeries
except ModuleNotFoundError:
    !pip install darts
    from darts import TimeSeries
from darts.models.forecasting.nbeats import NBEATSModel
from darts.metrics import mape

# For Georgia 

f = open(f"./datasets/weekly/GA/dataset.pkl", "rb")
df = pkl.load(f)
f = open(f"./datasets/weekly/GA/CDCdataset.pkl", "rb")
df2 = pkl.load(f)


df['date'] = pd.to_datetime(df['date'])

# Merge the datasets on date
df = df.merge(df2, on='date', how='inner')

df['date'] = pd.to_datetime(df['date'])

# Get the time series
tsy = TimeSeries.from_dataframe(df, 'date', target)
trainy = tsy[:-20]

def feature_selection_eval(features):
    tsx = TimeSeries.from_dataframe(df, 'date', features)

    # Get the training and testing data
    trainy = tsy[:-20]
    trainx = tsx[:-20]
    testy = tsy[-20:-10]
    testx = tsx[-20:-10]

    # Get the model
    model = NBEATSModel(input_chunk_length=20, output_chunk_length=10, n_epochs=100, random_state = 15)
    model.fit(trainy, past_covariates=trainx, val_series=testy, val_past_covariates=testx ,epochs = 100)
    predicted = model.predict(10)
    
    del model

    return np.array(predicted.values()), mape(predicted, testy)

# Get the features

allsyms = [col for col in df.columns if 'symptom' in col]

plt.figure(figsize=(20, 10))
plt.plot(np.array(tsy.values()), label='actual')
meths = {'pedicted_all':allsyms, 'predicted_granger':gransyms, 'predicted_pearson':pearsyms, 'predicted_rfe':svmsyms}
for key in meths.keys():
    pred, mapeval = feature_selection_eval(meths[key])
    print(mapeval)
    plt.plot(np.concatenate((np.array(trainy.values()),pred), axis = 0), label=key)
plt.xlabel('Weeks')
plt.ylabel('New Cases')
plt.savefig('feature_selection.png', dpi=300)
plt.legend()
plt.show()

### Two Cents after Feature Selection

From the graphs it's clear symptoms extracted based solely on the correlation do not warrant a better forecasting model. RFE and Granger prove to be more stable. 

With regards to the forecasting and based on the feature selection metric we will check the symptoms of nearby states to Georgia. 

Based on the map of Georgia, 

the 1-step neighbors are: Alabama, Florida, South Carolina, Tennessee and North Carolina

the 2-step neighbors are: Arkansas, Kentucky, Mississippi, Missouri and Virginia

In [ ]:
GA1neighbors = ['AL', 'FL', 'NC', 'SC', 'TN']
GA2neighbors = ['AR', 'KY', 'MS', 'MO', 'VA']

# Find the Granger Causality symptoms for the neighbors 
def get_granger_symptoms(state):
    # Load the data
    f = open(f"./datasets/weekly/{state}/dataset.pkl", "rb")
    df = pkl.load(f)
    f = open(f"./datasets/weekly/{state}/CDCdataset.pkl", "rb")
    df2 = pkl.load(f)

    df['date'] = pd.to_datetime(df['date'])

    # Merge the datasets on date
    df = df.merge(df2, on='date', how='inner')

    df['date'] = pd.to_datetime(df['date'])
    
    # Get the symptoms
    symptoms = [col for col in df.columns if 'symptom' in col]

    # Get the Granger Causality
    gransyms = []
    corrsyms = []
    for symptom in symptoms:
        # Normalize 
        grangerdat = df[['new_case', symptom]]
        for columns in grangerdat.columns:
            grangerdat[columns] = (grangerdat[columns] - grangerdat[columns].mean()) / grangerdat[columns].std()

        # Granger Test
        try:
            gran = grangercausalitytests(grangerdat[['new_case', symptom]], maxlag=2, verbose=False)
        except ValueError:
            continue
        # Correlation Test
        corr, _ = pearsonr(grangerdat[symptom], grangerdat['new_case'])

        if gran[1][0]['ssr_ftest'][1] < 0.001:
            gransyms.append(symptom)

        if corr > 0.2:
            corrsyms.append(symptom)
    
    return gransyms, corrsyms

# Get the Granger Causality Symptoms for Georgia
GAgranger, GAcorr = get_granger_symptoms('GA')

GAgrandf = pd.DataFrame({'state':'GA', 'granger_symptoms_shared':len(GAgranger), 'correlation_symptoms_shared':len(GAcorr)}, index=[0])

# Get the Granger Causality Symptoms for neighbors and find the shared symptoms
for state in GA1neighbors:
    grangersyms, correlationsyms = get_granger_symptoms(state)
    GAgrandf = GAgrandf.append({'state':state, 'granger_symptoms_shared':len(set(GAgranger).intersection(set(grangersyms))), 'correlation_symptoms_shared':len(set(GAcorr).intersection(set(correlationsyms))) }, ignore_index=True)

for state in GA2neighbors:
    grangersyms, correlationsyms = get_granger_symptoms(state)
    GAgrandf = GAgrandf.append({'state':state, 'granger_symptoms_shared':len(set(GAgranger).intersection(set(grangersyms))), 'correlation_symptoms_shared':len(set(GAcorr).intersection(set(correlationsyms))) }, ignore_index=True)

# Plot the results on US map using plotly
import plotly.express as px
fig = px.choropleth(GAgrandf, locations="state",locationmode="USA-states", color="granger_symptoms_shared", scope="usa", color_continuous_scale=px.colors.sequential.Blues)
fig.show()

In [ ]:
fig = px.choropleth(GAgrandf, locations="state",locationmode="USA-states", color="correlation_symptoms_shared", scope="usa", color_continuous_scale=px.colors.sequential.Blues)